In [1]:

import json

PTT_URL = 'https://www.ptt.cc'

In [2]:
import requests
from bs4 import BeautifulSoup


def get_web_page(url):
    resp = requests.get(url=url,
                        cookies={'over18': '1'})
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [6]:
def get_articles(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    #print(dom)
    # 上頁
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']
    #print('Prev url:',prev_url)

    articles = []
    divs = soup.find_all('div', 'r-ent')
    #print([d.text for d in divs ])
    for d in divs:

        if '[食記]' in d.find('div',class_='title').text.strip():
            #if d.find('div', class_='date').text.strip() == date:

            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)
                except ValueError:

                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            if d.find('a'):
                href = d.find('a')['href']
                title = d.find('a').text
                date = d.find('div', class_='date').text.strip()
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'date': date,
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })

    return articles, prev_url

In [7]:
def get_author_ids(posts, pattern):
    ids = set()
    for post in posts:
        if pattern in post['author']:
            ids.add(post['author'])
    return ids


In [10]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Food/index.html')
    if current_page:
        articles = []
        page_count = 0
        # 抓幾頁
        page_max = 30
        current_articles, prev_url = get_articles(current_page)

        while current_articles and (page_count < page_max):
            print('Round:', page_count)
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page)
            page_count = page_count + 1
            print('Next Round:', page_count)

        # print(get_author_ids(articles, '5566'))
        print('有', len(articles), '篇文章')
        for a in articles:
            #if int(a['push_count']) > threshold:
            print(a)
        with open('food_title.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

        # TODO: 抓文章內容

Round: 0
Next Round: 1
Round: 1
Next Round: 2
Round: 2
Next Round: 3
Round: 3
Next Round: 4
Round: 4
Next Round: 5
Round: 5
Next Round: 6
Round: 6
Next Round: 7
Round: 7
Next Round: 8
Round: 8
Next Round: 9
Round: 9
Next Round: 10
Round: 10
Next Round: 11
Round: 11
Next Round: 12
Round: 12
Next Round: 13
Round: 13
Next Round: 14
Round: 14
Next Round: 15
Round: 15
Next Round: 16
Round: 16
Next Round: 17
Round: 17
Next Round: 18
Round: 18
Next Round: 19
Round: 19
Next Round: 20
Round: 20
Next Round: 21
Round: 21
Next Round: 22
Round: 22
Next Round: 23
Round: 23
Next Round: 24
Round: 24
Next Round: 25
Round: 25
Next Round: 26
Round: 26
Next Round: 27
Round: 27
Next Round: 28
Round: 28
Next Round: 29
Round: 29
Next Round: 30
有 554 篇文章
{'date': '3/27', 'title': '[食記] 台中南區大慶 品克燒烤肉(外帶平價串燒烤', 'href': '/bbs/Food/M.1648391418.A.D60.html', 'push_count': 0, 'author': 'TW00500203'}
{'date': '3/27', 'title': '[食記] 台北車站 泰獅 泰式料理', 'href': '/bbs/Food/M.1648391699.A.48B.html', 'push_count': 0, 'author':